## BARRIER OPTIONS
This code builds a binomial model to study the behaviour of an up and out barrier call option. It collects user inputs, computes the risk neutral probability and then creates every possible sequence of up and down moves over the selected horizon. For each sequence it generates the full stock price path and checks whether the barrier is ever reached. If the barrier is hit the path payoff is set to zero, otherwise the payoff is the usual call value at expiry. The code attaches a probability to every path based on how many up and down moves it contains. It then forms the expected payoff by weighting each outcome with its probability and discounting it back to the present. The final printed result is the estimated price of the barrier option together with a few sample paths to show how the logic works.

In [3]:
# ============================================================
# 1. BARRIER OPTIONS (Up-and-Out Call)
# ============================================================
import numpy as np
from itertools import product

print("="*60)
print("BARRIER OPTION PRICING (Up-and-Out Call)")
print("="*60)

S0 = float(input("Enter initial stock price S0: "))
u = float(input("Enter up factor u (>1): "))
d = float(input("Enter down factor d (<1): "))
r = float(input("Enter risk-free rate r (decimal): "))
n = int(input("Enter number of time steps n: "))
K = float(input("Enter strike price K: "))
B = float(input("Enter barrier level B (option dies if S >= B): "))

q = (1 + r - d) / (u - d)
print(f"\nRisk-neutral probability q = {q:.4f}, 1-q = {1-q:.4f}")

paths = list(product([u, d], repeat=n))
price_paths = []
barrier_breached = []

for path in paths:
    prices = [S0]
    breached = False
    for move in path:
        new_price = prices[-1] * move
        if new_price >= B:
            breached = True
        prices.append(new_price)
    price_paths.append(prices)
    barrier_breached.append(breached)

payoffs = []
for i, p in enumerate(price_paths):
    if barrier_breached[i]:
        payoffs.append(0)
    else:
        payoffs.append(max(p[-1] - K, 0))

discount_factor = (1 / (1 + r)) ** n
V0 = 0

for i, path in enumerate(paths):
    up_moves = path.count(u)
    down_moves = path.count(d)
    prob = (q ** up_moves) * ((1 - q) ** down_moves)
    V0 += prob * payoffs[i]

V0 *= discount_factor

print(f"\nPrice of Barrier Option (Up-and-Out Call): {V0:.4f}")
print("\nSample paths (first 8):")
for i in range(min(8, len(price_paths))):
    status = "KNOCKED OUT" if barrier_breached[i] else "ACTIVE"
    print(f"Path {i+1}: Final={price_paths[i][-1]:.2f}, Status={status}, Payoff={payoffs[i]:.2f}")

BARRIER OPTION PRICING (Up-and-Out Call)


Enter initial stock price S0:  100
Enter up factor u (>1):  2
Enter down factor d (<1):  0.5
Enter risk-free rate r (decimal):  0.05
Enter number of time steps n:  2
Enter strike price K:  90
Enter barrier level B (option dies if S >= B):  110



Risk-neutral probability q = 0.3667, 1-q = 0.6333

Price of Barrier Option (Up-and-Out Call): 2.1063

Sample paths (first 8):
Path 1: Final=400.00, Status=KNOCKED OUT, Payoff=0.00
Path 2: Final=100.00, Status=KNOCKED OUT, Payoff=0.00
Path 3: Final=100.00, Status=ACTIVE, Payoff=10.00
Path 4: Final=25.00, Status=ACTIVE, Payoff=0.00


## CHOOSER OPTIONS
This code evaluates a chooser option in a binomial setting where the holder can later decide whether the contract becomes a call or a put. It starts by collecting the model parameters and computing the risk neutral probability, then builds every possible pattern of up and down movements over the full life of the option. For each pattern it constructs the corresponding price path and identifies the stock price at the choice time and at expiry. It computes both the call payoff and the put payoff at the final node and selects the larger value since the holder will take the more favourable side. The program attaches the appropriate risk neutral probability to each path and aggregates all payoffs into an expected value. It discounts this expected value back to the present to obtain the option price. The output includes the computed price along with a few sample paths that illustrate how the choice is made.

In [5]:
# ============================================================
# 2. CHOOSER OPTIONS
# ============================================================
print("\n" + "="*60)
print("CHOOSER OPTION PRICING")
print("="*60)

S0 = float(input("Enter initial stock price S0: "))
u = float(input("Enter up factor u (>1): "))
d = float(input("Enter down factor d (<1): "))
r = float(input("Enter risk-free rate r (decimal): "))
n = int(input("Enter total time steps n: "))
t_choose = int(input("Enter time step when choice is made (< n): "))
K = float(input("Enter strike price K: "))

q = (1 + r - d) / (u - d)
print(f"\nRisk-neutral probability q = {q:.4f}, 1-q = {1-q:.4f}")

paths = list(product([u, d], repeat=n))
price_paths = []

for path in paths:
    prices = [S0]
    for move in path:
        prices.append(prices[-1] * move)
    price_paths.append(prices)

payoffs = []
for p in price_paths:
    S_choose = p[t_choose]
    S_final = p[-1]
    
    call_value = max(S_final - K, 0)
    put_value = max(K - S_final, 0)
    
    chosen_payoff = max(call_value, put_value)
    payoffs.append(chosen_payoff)

discount_factor = (1 / (1 + r)) ** n
V0 = 0

for i, path in enumerate(paths):
    up_moves = path.count(u)
    down_moves = path.count(d)
    prob = (q ** up_moves) * ((1 - q) ** down_moves)
    V0 += prob * payoffs[i]

V0 *= discount_factor

print(f"\nPrice of Chooser Option: {V0:.4f}")
print(f"(Choice made at step {t_choose}, expires at step {n})")
print("\nSample paths (first 6):")
for i in range(min(6, len(price_paths))):
    S_choose = price_paths[i][t_choose]
    S_final = price_paths[i][-1]
    call_val = max(S_final - K, 0)
    put_val = max(K - S_final, 0)
    chosen = "CALL" if call_val >= put_val else "PUT"
    print(f"Path {i+1}: S@choice={S_choose:.2f}, Final={S_final:.2f}, Chose={chosen}, Payoff={payoffs[i]:.2f}")



CHOOSER OPTION PRICING


Enter initial stock price S0:  100
Enter up factor u (>1):  2
Enter down factor d (<1):  0.5
Enter risk-free rate r (decimal):  0.05
Enter total time steps n:  2
Enter time step when choice is made (< n):  1
Enter strike price K:  90



Risk-neutral probability q = 0.3667, 1-q = 0.6333

Price of Chooser Option: 65.6639
(Choice made at step 1, expires at step 2)

Sample paths (first 6):
Path 1: S@choice=200.00, Final=400.00, Chose=CALL, Payoff=310.00
Path 2: S@choice=200.00, Final=100.00, Chose=CALL, Payoff=10.00
Path 3: S@choice=50.00, Final=100.00, Chose=CALL, Payoff=10.00
Path 4: S@choice=50.00, Final=25.00, Chose=PUT, Payoff=65.00


## RAINBOW OPTIONS
This code prices a best-of-two rainbow call inside a two-asset binomial framework. It gathers initial prices, separate up and down factors for each asset, the risk free rate, the number of steps and the strike. It then computes a risk neutral probability and enumerates every sequence of up and down moves over the horizon. For each sequence it builds the full price path for both assets by applying the asset specific moves. At expiry it identifies the larger of the two final prices and sets the payoff to the positive difference between that best price and the strike. Each path payoff is weighted by its risk neutral probability, summed across all paths and discounted back to the present to produce the option value. Finally the code prints the estimated price alongside a few sample paths that show the final values, which asset won and the resulting payoff.

In [7]:
# ============================================================
# 3. RAINBOW OPTIONS (Best-of-Two Assets Call)
# ============================================================
print("\n" + "="*60)
print("RAINBOW OPTION PRICING (Best-of-Two Assets)")
print("="*60)

S1_0 = float(input("Enter initial price of Asset 1 (S1_0): "))
S2_0 = float(input("Enter initial price of Asset 2 (S2_0): "))
u1 = float(input("Enter up factor for Asset 1 (u1): "))
d1 = float(input("Enter down factor for Asset 1 (d1): "))
u2 = float(input("Enter up factor for Asset 2 (u2): "))
d2 = float(input("Enter down factor for Asset 2 (d2): "))
r = float(input("Enter risk-free rate r (decimal): "))
n = int(input("Enter number of time steps n: "))
K = float(input("Enter strike price K: "))

q = (1 + r - d1) / (u1 - d1)
print(f"\nRisk-neutral probability q = {q:.4f}, 1-q = {1-q:.4f}")
print("(Assuming same q for both assets)")

paths = list(product([u1, d1], repeat=n))
asset1_paths = []
asset2_paths = []

for path in paths:
    prices1 = [S1_0]
    prices2 = [S2_0]
    for i, move in enumerate(path):
        prices1.append(prices1[-1] * move)
        if move == u1:
            prices2.append(prices2[-1] * u2)
        else:
            prices2.append(prices2[-1] * d2)
    asset1_paths.append(prices1)
    asset2_paths.append(prices2)

payoffs = []
for i in range(len(asset1_paths)):
    best_asset = max(asset1_paths[i][-1], asset2_paths[i][-1])
    payoffs.append(max(best_asset - K, 0))

discount_factor = (1 / (1 + r)) ** n
V0 = 0

for i, path in enumerate(paths):
    up_moves = path.count(u1)
    down_moves = path.count(d1)
    prob = (q ** up_moves) * ((1 - q) ** down_moves)
    V0 += prob * payoffs[i]

V0 *= discount_factor

print(f"\nPrice of Rainbow Option (Best-of-Two Call): {V0:.4f}")
print("\nSample paths (first 6):")
for i in range(min(6, len(asset1_paths))):
    S1_final = asset1_paths[i][-1]
    S2_final = asset2_paths[i][-1]
    best = max(S1_final, S2_final)
    winner = "Asset 1" if S1_final >= S2_final else "Asset 2"
    print(f"Path {i+1}: S1={S1_final:.2f}, S2={S2_final:.2f}, Best={best:.2f} ({winner}), Payoff={payoffs[i]:.2f}")



RAINBOW OPTION PRICING (Best-of-Two Assets)


Enter initial price of Asset 1 (S1_0):  100
Enter initial price of Asset 2 (S2_0):  120
Enter up factor for Asset 1 (u1):  2
Enter down factor for Asset 1 (d1):  0.5
Enter up factor for Asset 2 (u2):  2
Enter down factor for Asset 2 (d2):  1
Enter risk-free rate r (decimal):  0.05
Enter number of time steps n:  5
Enter strike price K:  90



Risk-neutral probability q = 0.3667, 1-q = 0.6333
(Assuming same q for both assets)

Price of Rainbow Option (Best-of-Two Call): 377.7610

Sample paths (first 6):
Path 1: S1=3200.00, S2=3840.00, Best=3840.00 (Asset 2), Payoff=3750.00
Path 2: S1=800.00, S2=1920.00, Best=1920.00 (Asset 2), Payoff=1830.00
Path 3: S1=800.00, S2=1920.00, Best=1920.00 (Asset 2), Payoff=1830.00
Path 4: S1=200.00, S2=960.00, Best=960.00 (Asset 2), Payoff=870.00
Path 5: S1=800.00, S2=1920.00, Best=1920.00 (Asset 2), Payoff=1830.00
Path 6: S1=200.00, S2=960.00, Best=960.00 (Asset 2), Payoff=870.00


## SPREAD OPTIONS
This code values a call written on the price difference between two assets inside a two asset binomial setup. It begins by taking starting prices, the up and down factors for each asset, the risk free rate, the number of steps and the strike spread. It constructs every possible sequence of up and down moves and generates matching price paths for both assets over the full horizon. At the final time it computes the spread as the difference between the two terminal prices and applies the payoff rule which keeps only positive excess above the strike spread. Each payoff is multiplied by its risk neutral probability derived from the up and down counts along the path. The sum of all weighted payoffs is then discounted back to obtain today’s value of the spread call. The output also shows a few illustrative paths along with their terminal prices, the spread and the resulting payoff.

In [13]:

# ============================================================
# 4. SPREAD OPTIONS (Call on Spread)
# ============================================================
print("\n" + "="*60)
print("SPREAD OPTION PRICING (Call on S1 - S2)")
print("="*60)

S1_0 = float(input("Enter initial price of Asset 1 (S1_0): "))
S2_0 = float(input("Enter initial price of Asset 2 (S2_0): "))
u1 = float(input("Enter up factor for Asset 1 (u1): "))
d1 = float(input("Enter down factor for Asset 1 (d1): "))
u2 = float(input("Enter up factor for Asset 2 (u2): "))
d2 = float(input("Enter down factor for Asset 2 (d2): "))
r = float(input("Enter risk-free rate r (decimal): "))
n = int(input("Enter number of time steps n: "))
K = float(input("Enter strike spread K: "))

q = (1 + r - d1) / (u1 - d1)
print(f"\nRisk-neutral probability q = {q:.4f}, 1-q = {1-q:.4f}")
print("(Assuming same q for both assets)")

paths = list(product([u1, d1], repeat=n))
asset1_paths = []
asset2_paths = []

for path in paths:
    prices1 = [S1_0]
    prices2 = [S2_0]
    for i, move in enumerate(path):
        prices1.append(prices1[-1] * move)
        if move == u1:
            prices2.append(prices2[-1] * u2)
        else:
            prices2.append(prices2[-1] * d2)
    asset1_paths.append(prices1)
    asset2_paths.append(prices2)

payoffs = []
for i in range(len(asset1_paths)):
    spread = asset1_paths[i][-1] - asset2_paths[i][-1]
    payoffs.append(max(spread - K, 0))

discount_factor = (1 / (1 + r)) ** n
V0 = 0

for i, path in enumerate(paths):
    up_moves = path.count(u1)
    down_moves = path.count(d1)
    prob = (q ** up_moves) * ((1 - q) ** down_moves)
    V0 += prob * payoffs[i]

V0 *= discount_factor

print(f"\nPrice of Spread Option (Call on S1 - S2): {V0:.4f}")
print("\nSample paths (first 6):")
for i in range(min(6, len(asset1_paths))):
    S1_final = asset1_paths[i][-1]
    S2_final = asset2_paths[i][-1]
    spread = S1_final - S2_final
    print(f"Path {i+1}: S1={S1_final:.2f}, S2={S2_final:.2f}, Spread={spread:.2f}, Payoff={payoffs[i]:.2f}")



SPREAD OPTION PRICING (Call on S1 - S2)


Enter initial price of Asset 1 (S1_0):  100
Enter initial price of Asset 2 (S2_0):  120
Enter up factor for Asset 1 (u1):  2
Enter down factor for Asset 1 (d1):  0.5
Enter up factor for Asset 2 (u2):  2
Enter down factor for Asset 2 (d2):  1
Enter risk-free rate r (decimal):  0.05
Enter number of time steps n:  7
Enter strike spread K:  110



Risk-neutral probability q = 0.3667, 1-q = 0.6333
(Assuming same q for both assets)

Price of Spread Option (Call on S1 - S2): 0.0000

Sample paths (first 6):
Path 1: S1=12800.00, S2=15360.00, Spread=-2560.00, Payoff=0.00
Path 2: S1=3200.00, S2=7680.00, Spread=-4480.00, Payoff=0.00
Path 3: S1=3200.00, S2=7680.00, Spread=-4480.00, Payoff=0.00
Path 4: S1=800.00, S2=3840.00, Spread=-3040.00, Payoff=0.00
Path 5: S1=3200.00, S2=7680.00, Spread=-4480.00, Payoff=0.00
Path 6: S1=800.00, S2=3840.00, Spread=-3040.00, Payoff=0.00
